<a href="https://colab.research.google.com/github/mahmouddraz/coffee_breaks/blob/main/coffee_breaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Upload an excel/csv contains the employees's names, emails

In [ ]:
from google.colab import files
names_list = files.upload()

# Enter you email login data 

In [ ]:
user_name = "hr@me.me
password = "password"

# Import packges

In [ ]:
pip install icalendar 

In [ ]:
pip install openpyxl==3.0.9

In [ ]:
import random
import itertools
import pandas as pd
import numpy as np 
import calendar
import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from icalendar import Calendar, Event
import datetime
import pytz

# How to use the tool

1. Enter your login details in the top cell and the password of the email you want to send from. 
2. Select from the Runetime menu and select "Run All". That's it!

# Get all possible combinations from the name list.

In [ ]:
def get_combinations(names):
    '''
    Get all combinations of names
    '''
    combinations = list(itertools.combinations(names, 2))
    return combinations

# Find the all possible pairs for a certain person

In [ ]:
def find_person_pair(combinations, name):
    '''
    Find all combination for certain emolopyee
    '''
    person_pairs_list = []

    for pair in combinations:

        if name in pair:
            person_pairs_list.append(pair)
            
    return person_pairs_list

In [ ]:
# remove non-selected combinations 
def remove_list_items(list_items, items_to_remove):
    return [item for item in list_items if item not in items_to_remove]

# Sample form the pairs list with a uniform distribution

In [ ]:
def random_sampling(combinations, name_list):
    '''
    Takes a list of strings and returns a list of random samples of those strings
    '''
    sample_list = []
    while len(combinations) > 0:
      for name in name_list:
        
        person1_pairs_list = find_person_pair(combinations, name)
        if len(person1_pairs_list) == 0: 
          continue
        sample = random.choice(person1_pairs_list)

        sample_list.append(sample)
        
        person2_pairs_list = find_person_pair(combinations, sample[1])
        person2_pairs_list.remove(sample)

        combinations = remove_list_items(combinations, person1_pairs_list+person2_pairs_list)
        if len(combinations) == 0:
          break 
    return sample_list

# Send email to the coffee mates with a calender

In [ ]:

def create_icalendar(summary, description, location, start_time, end_time):
    cal = Calendar()
    cal.add('prodid', '-//My calendar product//mxm.dk//')
    cal.add('version', '2.0')
    event = Event()
    event.add('summary', summary)
    event.add('description', description)
    event.add('location', location)
    event.add('dtstart', start_time)
    event.add('dtend', end_time)
    event.add('dtstamp', datetime.datetime.now())
    event['uid'] = '20050115T101010/27346262376@mxm.dk'
    event.add('priority', 5)
    cal.add_component(event)
    return cal

In [ ]:
'''
create an icalendar even and then send the icalendar as an attachment to an email in a TLS session 
'''

def send_email(user_name, password, subject, body, attachment, pairs_list, names_df):
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()
    server.login(user_name, password)
    
    for pair in pairs_list:
      for name in pair:
        msg = MIMEMultipart()
        msg['Subject'] = subject
        msg['From'] = user_name


        recipient_email = names_df['E-Mail'][names_df.Vorname==name].to_list()[0]
        mate_name = list(pair)
        mate_name.remove(name)
        mate_email = names_df['E-Mail'][names_df.Vorname==mate_name[0]].to_list()[0]

        msg['To'] = recipient_email
        msg.attach(MIMEText(body.format(name, mate_name[0], mate_email)))

        with open(attachment, "rb") as f:
          msg.attach(MIMEApplication(
              f.read(),
              Content_Disposition='attachment; filename="%s"' % attachment,
              Name=attachment
          ))

        server.sendmail(user_name, recipient_email, msg.as_string())

    server.quit()

# Main

In [ ]:
def main(user_name, password, names_list):

    summary = "Your Coffe Date"
    description = "Stay connected with your Hubject colleagues"
    location = "Teams online"

    time_now = datetime.datetime.now().replace(microsecond=0, second=0, hour=11, minute=30)
    start_time = time_now + datetime.timedelta(days=1)
    end_time = start_time + datetime.timedelta(minutes=20)

    cal = create_icalendar(summary, description, location, start_time, end_time)

    # import data from google drive 
    names_df = names_list

    # all possibel combinations
    combinations = get_combinations(names_df["Vorname"].tolist())
    # sample with unifomr distirbution
    pairs_list = random_sampling(combinations, names_df["Vorname"].tolist())
     # choose the date as string 
    date = datetime.date.today().isoformat()

    body = '''
    Hey {}, 

    Your Coffee date for the week with {}. And this is their Email {}. 

    Best, 
    Your hr team
    '''
    with open("meeting.ics", "wb") as f:
        f.write(cal.to_ical())
        
    send_email(user_name, password, "Your Coffee Break MAAAAATCH for this week_" + date, body, "meeting.ics", pairs_list, names_df)
        

# Run the code 

In [ ]:
if __name__ == "__main__":
  main(user_name, password, names_list)